## Non-negative Matrix Factorization

In [1]:
import json
import os

dirpath = os.getcwd() + '\\nouns_adj.json'

with open(dirpath, 'r') as f:
    documents = json.load(f)
    texts = [' '.join(doc) for doc in documents]

In [2]:
import pandas as pd

# Convert to DataFrame
df = pd.DataFrame(texts, columns=['headline_text'])
df.head()

,headline_text
0,март год полный солнечный_затмение полный фаза...
1,сентябрь полный лунный_затмение европейский ча...
2,март год полный солнечный_затмение полный фаза...
3,июль год продолжительный нынешний столетие пол...
4,первый упоминание солнечный_затмение год эра д...


### Filter out tokens with TF-IDF

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Use TF-IDF by removing tokens that appear in more than 20% of the documents
# and in less than 1% of the documents
vect = TfidfVectorizer(min_df=.01, max_df=.2)

# Fit and transform
X = vect.fit_transform(df.headline_text)

### Build an NMF model

In [4]:
from sklearn.decomposition import NMF

# Create an NMF instance
nmf_model = NMF(n_components=20, random_state=5, max_iter=1500)
 
# Fit the model to TF-IDF
nmf_model.fit(X)
 
# Transform the TF-IDF
nmf_features = nmf_model.transform(X)

C:\Users\satan\AppData\Roaming\Python\Python39\site-packages\sklearn\decomposition\_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


In [5]:
# Create a dataframe
components_df = pd.DataFrame(nmf_model.components_, columns=vect.get_feature_names())
# components_df

### Print the results

In [6]:
# Print the results
for topic in range(components_df.shape[0]):
    tmp = components_df.iloc[topic]
    print(f'For topic {topic+1} the words with the highest value are:' +
          f'\n{tmp.nlargest(15)}\n')

For topic 1 the words with the highest value are:
губка                      1.729937
личинка                    1.280359
билатерия                  1.056967
позвоночный                1.054618
многоклеточный_животное    1.030132
гребневик                  0.990422
hox                        0.969017
членистоногий              0.941092
ветвь                      0.839551
нервный_система            0.828146
общий_предок               0.816278
двусторонне                0.761883
эволюционный               0.710134
рот                        0.692143
предок                     0.682257
Name: 0, dtype: float64

For topic 2 the words with the highest value are:
самец           1.894047
самка           1.665771
спаривание      0.250543
половой         0.207411
ухаживание      0.203908
паук            0.191130
потомство       0.153306
сперматозоид    0.153138
песня           0.149903
пение           0.149719
особь           0.145954
выбор           0.145090
дрозофила       0.132450
брачный   

### Explore a certain document

In [7]:
# Look at the nth document
n = 3
my_document = df.headline_text[n]
my_document

'июль год продолжительный нынешний столетие полный солнечный_затмение максимальный длительность полный фаза минута_секунда путь лунный_тень всемирный время западный побережье индия китай кульминация юго-восточный японский_остров второй_половина путь безбрежный простор тихий_океан тень небольшой атолл полный затмение житель юго-восточный_азия россия снг максимальный_фаза затмение приморье таджикистан июль год полный солнечный_затмение больший продолжительность полный фаза нынешний столетие повторение сарос цикл повторение затмение равный день год день полный солнечный_затмение июль год полоса полный фаза затмение вода тихий_океан страна центральный южный_америка закрытый луна диск солнце год житель юго-восточный_азия россия снг максимальный_фаза затмение приморье таджикистан путь лунный_тень земля час гринвичский полуночь июль всемирный время западный побережье индия всемирный час час лето луна время перигей место июль видимый размер лунный_диск лунный_тень поверхность земля близкий бол

In [8]:
# Get the topic number for the document
pd.DataFrame(nmf_features).loc[n].idxmax() + 1

12

### The number of documents for each topic 

In [9]:
pd.DataFrame(nmf_features).idxmax(axis=1).value_counts()

15    149
10    117
5      87
8      80
13     77
14     65
3      64
7      63
1      63
18     62
9      58
4      55
17     52
16     50
2      49
0      46
12     43
19     38
6      27
11     15
dtype: int64